In [ ]:
from pydub import AudioSegment
from pydub.effects import normalize
import math
from urllib import request
import joblib
from huggingsound import SpeechRecognitionModel
import os
# import noisereduce as nr
import IPython
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np
import io
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import gc

In [ ]:
sound=AudioSegment.from_wav("Y2022_M3_D19_H20_M58_rand753131_5.wav")

In [ ]:
sound = sound.set_frame_rate(16000)

In [ ]:
sound[0:40*1000].export("teste.wav",format="wav")

In [ ]:
sound[0:40*1000]

In [ ]:
model = joblib.load('model.joblib')

In [ ]:
transcription = model.transcribe(["teste.wav"])

In [ ]:
sum(transcription[0]['probabilities'])/len(transcription[0]['probabilities'])

In [ ]:
raw_transc=transcription[0]['transcription']
raw_transc

In [ ]:
rate, data = wavfile.read("teste.wav")
# perform noise reduction
reduced_noise = nr.reduce_noise(y=data, sr=rate,n_jobs=-1,prop_decrease=1)
wavfile.write("teste2.wav", rate, reduced_noise)
fig, ax = plt.subplots(figsize=(20,3))
ax.plot(reduced_noise-5)

In [ ]:
sound=AudioSegment.from_wav("teste2.wav")

In [ ]:
sound+30
sound.export("teste2.wav",format="wav")
sound

In [ ]:
transcription = model.transcribe(["teste2.wav"])

In [ ]:
sum(transcription[0]['probabilities'])/len(transcription[0]['probabilities'])

In [ ]:
transcription[0]['transcription']

In [ ]:
raw_transc

Criando função que não da load no modelo sempre

In [ ]:
def convert_audio(audio_path,filename):
    local_file = filename
    request.urlretrieve(audio_path, local_file)
    sound = AudioSegment.from_ogg(local_file)
    sound = sound.set_frame_rate(16000)
    sound=normalize(sound)
    audiolen=20
    i=0
    text=""
    probsum=[]
    problen=[]
    subitems=math.ceil(len(sound)/60000*60/audiolen)
    
    if(subitems>0):
        for item in range(0,subitems):
            if(item+1==math.ceil(len(sound)/60000*60/50)):
                start=i*audiolen*1000
                end=len(sound)/60000*60*1000 
            else:
                start=audiolen*i*1000
                end=audiolen*(i+1)*1000
            sound[start:end].export(f"{filename[:-4]}_{i}.wav", format="wav")
            i+=1
    else:
            sound[start:end].export(f"{filename[:-4]}.wav", format="wav")

    model = joblib.load('model.joblib')
    #model = load_model()
    
    i=0
    if(subitems>0):
        for item in range(0,subitems):
            transcription = model.transcribe([f"{filename[:-4]}_{i}.wav"])
            text += transcription[0]['transcription']
            probsum.append(sum(transcription[0]['probabilities']))
            problen.append(len(transcription[0]['probabilities']))
            os.remove(f"{filename[:-4]}_{i}.wav")
            i+=1
    else:
        transcription = model.transcribe([f"{filename[:-4]}.wav"])
        text += transcription[0]['transcription']
        probsum.append(sum(transcription[0]['probabilities']))
        problen.append(len(transcription[0]['probabilities']))
        os.remove(f"{filename[:-4]}.wav")
    
    probability=sum(probsum)/sum(problen)
    
    return text,probability


In [ ]:
convert_audio("https://firebasestorage.googleapis.com/v0/b/hear4me-e0481.appspot.com/o/audio%2FY2022_M3_D19_H20_M58_rand753131.ogg?alt=media&token=6b4f907f-ad99-4ae9-887a-57278e6e911b","Y2022_M3_D19_H20_M58_rand753131.ogg")

In [ ]:
# pip install noisereduce

In [ ]:
import noisereduce as nr
import IPython
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np
import io
# load data
rate, data = wavfile.read("teste.wav")
# perform noise reduction
reduced_noise = nr.reduce_noise(y=data, sr=rate)

In [ ]:
IPython.display.Audio(data=data, rate=rate)

In [ ]:
fig, ax = plt.subplots(figsize=(20,3))
ax.plot(data)

In [ ]:
reduced_noise = nr.reduce_noise(y=data, sr=rate, n_std_thresh_stationary=0.5,stationary=True)
reduced_noise = nr.reduce_noise(y=reduced_noise, sr=rate, thresh_n_mult_nonstationary=0.5,stationary=False)
wavfile.write("teste2.wav", rate, reduced_noise)
fig, ax = plt.subplots(figsize=(20,3))
ax.plot(reduced_noise)

In [ ]:
fig, ax = plt.subplots(figsize=(20,3))
ax.plot(reduced_noise)

In [ ]:
IPython.display.Audio(data=reduced_noise, rate=rate)

Testando somente a função de criação de audios + Huggingfaces

In [ ]:
def convert_audio(audio_path,filename):
    local_file = filename
    request.urlretrieve(audio_path, local_file)
    sound = AudioSegment.from_ogg(local_file)
    sound = sound.set_frame_rate(16000)
    sound=normalize(sound)
    audiolen=5
    i=0
    text=""
    probsum=[]
    problen=[]
    subitems=math.ceil(len(sound)/60000*60/audiolen)
    
    if(subitems>0):
        for item in range(0,subitems):
            if(item+1==math.ceil(len(sound)/60000*60/50)):
                start=i*audiolen*1000
                end=len(sound)/60000*60*1000 
            else:
                start=audiolen*i*1000
                end=audiolen*(i+1)*1000
            sound[start:end].export(f"{filename[:-4]}_{i}.wav", format="wav")
            i+=1
    else:
            sound[start:end].export(f"{filename[:-4]}.wav", format="wav")
    
    model=joblib.load('model.joblib')
    
    i=0
    if(subitems>0):
        for item in range(0,subitems):
#             if(i!=0 & i%4==0):
#                 del model
#                 gc.collect()
#                 model=joblib.load('model.joblib')
                
            transcription = model.transcribe([f"{filename[:-4]}_{i}.wav"])
            text += transcription[0]['transcription']
            probsum.append(sum(transcription[0]['probabilities']))
            problen.append(len(transcription[0]['probabilities']))
            os.remove(f"{filename[:-4]}_{i}.wav")
            i+=1
            print(text)
    else:
        transcription = model.transcribe([f"{filename[:-4]}.wav"])
        text += transcription[0]['transcription']
        probsum.append(sum(transcription[0]['probabilities']))
        problen.append(len(transcription[0]['probabilities']))
        os.remove(f"{filename[:-4]}.wav")
    
    probability=sum(probsum)/sum(problen)
    
    return text,probability


In [ ]:
convert_audio("https://firebasestorage.googleapis.com/v0/b/hear4me-e0481.appspot.com/o/audio%2FY2022_M3_D19_H20_M58_rand753131.ogg?alt=media&token=6b4f907f-ad99-4ae9-887a-57278e6e911b","Y2022_M3_D19_H20_M58_rand753131.ogg")

In [1]:
# Installing Transformer

!pip install -q transformers

In [2]:
# Import necessary library

# For managing audio file
import librosa

#Importing Pytorch
import torch

#Importing Wav2Vec
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

2022-04-20 17:13:41.081369: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-20 17:13:41.081402: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import IPython.display as display
display.Audio("teste2.wav", autoplay=True)

In [4]:
audio, rate = librosa.load("teste.wav", sr = 16000)

In [5]:
print(audio)

[0.00218196 0.00380863 0.00324665 ... 0.00759719 0.00551351 0.00447233]


In [6]:
print(rate)

16000


In [7]:
# Importing Wav2Vec pretrained model
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/home/luisfelipet/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:746: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Taking an input value

input_values = tokenizer(audio, return_tensors = "pt").input_values

In [9]:
# Storing logits (non-normalized prediction values)
logits = model(input_values).logits

In [10]:
# Storing predicted ids
prediction = torch.argmax(logits, dim = -1)

In [11]:
# Passing the prediction to the tokenzer decode to get the transcription
transcription = tokenizer.batch_decode(prediction)[0]

In [12]:
print(transcription)

TE WEEFELL ITO PAGAMETO WAS IS WOES PORDAO CHES PASSESSETA DIS A A TON'T POCOWISE POC E GINA TO SER BALCHAFALONE E MORDOCORO ITICATE TE OSESELF WE COLROGEY I AINT A DILL I IN SY GID I EVILL SA NE AN MO TO PUT FOR NENSIEUR DE RAI I EVIL SAY THE CHALL DIPI TE COPRADI MIPACONPRADUTE FOR ESE DE MA BARAD


In [13]:
del tokenizer
del model

In [14]:
import gc

In [16]:
gc.collect()

0

In [1]:
from pydub import AudioSegment
from pydub.effects import normalize
import math
from urllib import request
import joblib
from huggingsound import SpeechRecognitionModel
import os
# import noisereduce as nr
import IPython
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np
import io
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import gc
import time

In [2]:
import multiprocessing as mp

In [3]:
print("Number of processors: ", mp.cpu_count())

Number of processors:  12


In [8]:
def convert_audio(audio_path, filename):
    local_file = filename
    request.urlretrieve(audio_path, local_file)
    sound = AudioSegment.from_ogg(local_file)
    sound = sound.set_frame_rate(16000)
    sound = normalize(sound)
    audiolen = 5
    i = 0
    text = ""
    probsum = []
    problen = []
    subitems = math.ceil(len(sound) / 60000 * 60 / audiolen)

    if (subitems > 0):
        for item in range(0, subitems):
            if (item + 1 == subitems):
                start = i * audiolen * 1000
                end = len(sound) / 60000 * 60 * 1000
            else:
                start = audiolen * i * 1000
                end = audiolen * (i + 1) * 1000
            sound[start:end].export(f"{filename[:-4]}_{i}.wav", format="wav")
            i += 1
    else:
        sound[start:end].export(f"{filename[:-4]}.wav", format="wav")

    model = joblib.load('model.joblib')
    os.remove(local_file)
    if (subitems > 0):
        for item in range(0, subitems):
            transcription = model.transcribe([f"{filename[:-4]}_{item}.wav"])
            text += transcription[0]['transcription']
            probsum.append(sum(transcription[0]['probabilities']))
            problen.append(len(transcription[0]['probabilities']))
            os.remove(f"{filename[:-4]}_{item}.wav")
    else:
        transcription = model.transcribe([f"{filename[:-4]}.wav"])
        text += transcription[0]['transcription']
        probsum.append(sum(transcription[0]['probabilities']))
        problen.append(len(transcription[0]['probabilities']))
        os.remove(f"{filename[:-4]}.wav")
    
    probability = sum(probsum) / sum(problen)

    return text, probability

In [9]:
# pool = mp.Pool(mp.cpu_count())

Paralelizando o processamento

In [10]:
start_time = time.time()
text,proba=convert_audio("https://firebasestorage.googleapis.com/v0/b/hear4me-e0481.appspot.com/o/audio%2FY2022_M3_D19_H22_M8_rand80940.ogg?alt=media&token=d859ffdf-4742-4e83-b82d-ba1c3a3fc957","Y2022_M3_D19_H22_M8_rand80940.ogg")
print("--- %s seconds ---" % (time.time() - start_time))
print(text)
print(proba)


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.30it/s]

--- 9.279735326766968 seconds ---
o que eu acho bem interessantes sobre pássaros é que eles são sempre animaiscom pontas de asas coloridas e eles também tendem a teruma peninha muito interessante em cima da cabeça as peninhas na cabeçadois pombos em geral são roxas ou verdes mas nadamais importa somente o que a vida pode trazerde felicidade e de i reverência
0.9792152116635094


In [11]:
def transcribe(model, item,filename):
    transcription = model.transcribe([f"{filename[:-4]}_{item}.wav"])
    os.remove(f"{filename[:-4]}_{i}.wav")
    return transcription[0]['transcription'], sum(transcription[0]['probabilities']), len(transcription[0]['probabilities'])

def collect_result(result):
    global results
    results.append(result)

In [12]:
def convert_audio(audio_path, filename):
    local_file = filename
    request.urlretrieve(audio_path, local_file)
    sound = AudioSegment.from_ogg(local_file)
    sound = sound.set_frame_rate(16000)
    sound = normalize(sound)
    audiolen = 5
    i = 0
    text = ""
    probsum = []
    problen = []
    subitems = math.ceil(len(sound) / 60000 * 60 / audiolen)

    if (subitems > 0):
        for item in range(0, subitems):
            if (item + 1 == subitems):
                start = i * audiolen * 1000
                end = len(sound) / 60000 * 60 * 1000
            else:
                start = audiolen * i * 1000
                end = audiolen * (i + 1) * 1000
            sound[start:end].export(f"{filename[:-4]}_{i}.wav", format="wav")
            i += 1
    else:
        sound[start:end].export(f"{filename[:-4]}.wav", format="wav")

    model = joblib.load('model.joblib')
    os.remove(local_file)
    if (subitems > 0):
        pool = mp.Pool(mp.cpu_count())
        for item in range(0, subitems):
            pool.apply_async(transcribe,args=(model,item,filename), callback=collect_result)
            pool.close()
            pool.join()
        transcribe(model,item)
    else:
        transcription = model.transcribe([f"{filename[:-4]}.wav"])
        text += transcription[0]['transcription']
        probsum.append(sum(transcription[0]['probabilities']))
        problen.append(len(transcription[0]['probabilities']))
        os.remove(f"{filename[:-4]}.wav")
    
    probability = sum(probsum) / sum(problen)

    return text, probability

In [14]:
start_time = time.time()
text,proba=convert_audio("https://firebasestorage.googleapis.com/v0/b/hear4me-e0481.appspot.com/o/audio%2FY2022_M3_D19_H22_M8_rand80940.ogg?alt=media&token=d859ffdf-4742-4e83-b82d-ba1c3a3fc957","Y2022_M3_D19_H22_M8_rand80940.ogg")
print("--- %s seconds ---" % (time.time() - start_time))
print(text)
print(proba)

  0%|                                                                                             | 0/1 [00:00<?, ?it/s]Process ForkPoolWorker-10:
Process ForkPoolWorker-12:
Process ForkPoolWorker-5:
Process ForkPoolWorker-4:
Process ForkPoolWorker-8:
Process ForkPoolWorker-3:
Process ForkPoolWorker-13:
Process ForkPoolWorker-6:
Process ForkPoolWorker-7:
Process ForkPoolWorker-11:
Process ForkPoolWorker-9:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/luisfelipet/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/luisfelipet/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/luisfelipet/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/luisfelipet/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/luisfelipet/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/luisfelipet/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/luisfelipet/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _boo

In [75]:
pool = mp.Pool(mp.cpu_count())
pool

<multiprocessing.pool.Pool state=RUN pool_size=12>

In [76]:
def multiply (i,a,b):
    return (i,a*b)

In [77]:
def get_result(result):
    global results
    results.append(result)

In [ ]:
start_time = time.time()
for i in range(0,10000000):
    pool.apply_async(multiply,args=(i,i,i+1),callback=get_result)
pool.close()
pool.join()
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
start_time = time.time()
for i in range(0,10000000):
    multiply(i,i,i+1)
print("--- %s seconds ---" % (time.time() - start_time))